In [1]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
import openai
import os
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from uuid import uuid4
import hashlib

In [2]:
df = pd.read_csv('Retail.OrderHistory.1.csv')

In [3]:
# Strip milliseconds from datetime
df['Order Date'] = df['Order Date'].str.replace(r'\.\d+', '', regex=True)

df['Order Date'] = pd.to_datetime(df['Order Date'], utc=True)


In [4]:
df_brief = df[['Order Date', 'Unit Price', 'Product Name', 'ASIN']]

In [5]:
df_small = df.sample(n=100)
#df_small = df.head(5)

In [6]:
df_brief_small = df_small[['Order Date', 'Unit Price', 'Product Name', 'ASIN']]

In [7]:
description_column = 'Product Name' 
asin_column = 'ASIN'
order_date_column = 'Order Date'

In [8]:
#df = df_brief_small

In [9]:
if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY is ready


In [10]:
# Use OpenAI embeddings for vectorization
openai_embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [11]:
# Initialize Chroma client
vector_store = Chroma(collection_name='amazon_products',persist_directory='./vectordb',embedding_function=openai_embedding)


In [29]:
# Generate a unique ID by hashing 'Order Date', 'Unit Price', and 'Product Name'
def generate_unique_id(row):
    unique_string = f"{row['Order Date']}_{row['Unit Price']}_{row['Product Name']}_{row['ASIN']}"
    return hashlib.md5(unique_string.encode()).hexdigest()


In [30]:
# Populate the vectore store
documents = []
uuids = []  # List to store UUIDs

# Loop through each row and use the generated unique ID
for index, row in df_to_store.iterrows():
    unique_id = generate_unique_id(row)
    description = row[description_column]
    
    if pd.notna(description):  # Ensure the description is not NaN
        
        # Create a Document object for each row
        document = Document(
            page_content=description,
            metadata={"Order Date": str(row['Order Date']), "Unit Price": row['Unit Price'], "Product Name": row['Product Name'], "ASIN": row['ASIN'], "Category": row['Category'], "Room": row['Room']},
            id=unique_id
        )
        
        # Add the document to the documents list
        documents.append(document)
        
        # Generate a UUID and add to the UUIDs list
        uuids.append(str(uuid4()))

# Add the documents to the vector store in bulk
vector_store.add_documents(documents=documents, ids=uuids)

['fd08ba9a-39e2-4287-99f5-bbc29b213c14',
 'e322c223-b81e-4476-9030-540dfdc06553',
 'd4f2b9c7-4f33-445f-a6a9-c731f079882c',
 '04d06b7a-113b-47cd-a596-a94f404bef5f',
 'cb18fd1b-4b03-4223-8a92-6a7a4a9cfb01',
 '1d4d570d-efee-489a-8710-8a3cbb085e94',
 '552ddb38-15a2-45c8-9474-41ab7d0ad303',
 '24da193c-d3b5-4ecd-8a5f-d33757ad4245',
 'c247cd96-cea0-4339-9f86-43fb88e2baf4',
 '12d2f983-0595-4c79-84a5-405303a8af17',
 '9ca4d787-ba12-433f-9828-93ca3416276f',
 'bd130c9e-89c9-4066-a226-45b4b2c877c9',
 'e96eae57-33bc-415d-ae01-3a4cc5314885',
 '50f238f0-ff90-44fc-bfcf-07d467e51197',
 'ef946526-3a80-4b36-b4fb-ccc632232007',
 'b2b8a38e-ddeb-45ca-ac1e-e961b62425aa',
 '7b20ece1-9c61-4a76-bf17-a18cd13b603e',
 'df52a0cb-1f0e-435a-8827-9f9072b1f16b',
 '72d6a377-9d78-4b95-a128-5064bc5db786',
 '691f88b5-f84c-4168-b970-fba813d6c03d',
 'db2b1b3a-37c3-4ec8-9c50-ad89c731c8dc',
 '623336ca-ef23-4042-8e34-95c4f077d91a',
 'cb9e8822-1e7e-4eba-95f6-0ab23b7b21aa',
 '34148e01-6964-41f4-b701-137732c8479e',
 'a7e1cd9b-5232-

In [12]:
def classify_product(product_descr):
    messages = [
        (
            "system",
            "You are a utility function in a python application program. Your job is to take a product description and return a 1 or 2 word category for the product along with 1 or 2 room names where you would find this product.",
        ),
        ("human", product_descr),
    ]
    response = llm.invoke(messages)
    return response.content

In [46]:
def query_orders(text, num_results = 1):
    results = vector_store.similarity_search(
    text,
    k=num_results,
    #filter={"Order Date":{"$gte": start_of_2024}},
)

    # Collect results in a list of dictionaries
    data = []

    for res in results:
        data.append({
            "Product Description": res.page_content,
            "Order Date": res.metadata.get('Order Date'),
            "Product Name": res.metadata.get('Product Name'),
            "Unit Price": res.metadata.get('Unit Price'),
            "ASIN": res.metadata.get('ASIN')
        })

    # Convert the list of dictionaries to a DataFrame
    df_results = pd.DataFrame(data)
    
    for index, row in df_results.iterrows():
        #print(f"Product Description: {row['Product Description']}")
        print(f"Product Name: {row['Product Name']}"),
        print(f"Order Date: {row['Order Date']}")
        print(f"ASIN: {row['ASIN']}"),
        print(classify_product(row['Product Name'])),
        print(f"Unit Price: {row['Unit Price']}\n")        
    return

In [61]:
query_text = "Clothes for sailing"
query_orders(query_text, 3)

Product Name: WindRider 3/4 Finger Performance Sailing Gloves
Order Date: 2021-07-11T15:59:26Z
ASIN: B06XKPZHXL
Category: Sailing Gear  
Rooms: Dock, Marina
Unit Price: 18.99

Product Name: WindRider 3/4 Finger Performance Sailing Gloves - Kayak Canoe Paddling
Order Date: 2021-07-11T15:58:13Z
ASIN: B014TH8J8O
Category: Sailing Gloves  
Room: Outdoors, Sports
Unit Price: 18.99

Product Name: MARCHWAY Floating Waterproof Dry Bag 5L/10L/20L/30L, Roll Top Sack Keeps Gear Dry for Kayaking, Rafting, Boating, Swimming, Camping, Hiking, Beach, Fishing (Bright Yellow, 10L)
Order Date: 2021-07-12T15:54:42Z
ASIN: B01N116D0C
Category: Dry Bag  
Rooms: Outdoors, Water Activities
Unit Price: 14.99



In [13]:
from langchain_openai import ChatOpenAI

# Initialize OpenAI client
llm = ChatOpenAI(model="gpt-4o-mini")

In [31]:
print(response.content)

Category: Bathroom Cleaner  
Room: Bathroom, Toilet


In [14]:
from typing import Optional

from pydantic import BaseModel, Field

In [15]:
class OrderItem(BaseModel):
    category: str = Field(description="The category of the item (home goods, bath supplies, sports equipment, etc)")
    room: str = Field(description="The room where this item is most likely to be found")


In [16]:
structured_llm = llm.with_structured_output(OrderItem)

In [17]:
item_response = structured_llm.invoke("MARCHWAY Floating Waterproof Dry Bag 5L/10L/20L/30L, Roll Top Sack Keeps Gear Dry for Kayaking, Rafting, Boating, Swimming, Camping, Hiking, Beach, Fishing (Bright Yellow, 10L)")

In [19]:
item_response.room

'outdoor'

In [20]:
item_response.category

'sports equipment'

In [21]:
df_brief_small.head()

,Order Date,Unit Price,Product Name,ASIN
122,2023-12-31 17:21:27+00:00,189.99,"Cricut Joy Machine Rainbow Permanent Vinyl, St...",B0B7XS1ZR5
9,2024-08-29 12:53:41+00:00,39.97,"Ultra Concentrated Organic Mushroom Coffee 3,5...",B0D41N4J1Y
993,2018-06-05 14:35:10+00:00,12.93,Tom's of Maine Natural Long Lasting Men's Deod...,B00LITYJYU
1373,2011-02-11 01:23:52+00:00,33.36,UTG Covert Homeland Security Gun Case (32-Inch),B001WJ6O9E
149,2023-11-24 22:45:19+00:00,16.99,"Amazon Basics Dishwasher Detergent Pacs, Fresh...",B09CLR75K6


In [22]:
start_date = pd.to_datetime('2020-01-01', utc=True)
end_date = pd.to_datetime('2024-10-01', utc=True)

df_filtered = df_brief[(df_brief['Order Date'] >= start_date) & (df_brief['Order Date'] < end_date)]


In [23]:
df_new = df_filtered.copy()
df_new['Category'] = None
df_new['Room'] = None

In [24]:
for index, row in df_new.iterrows():
    description = row[description_column]
    item_response = structured_llm.invoke(description)
    category = item_response.category
    room = item_response.room
    #print(f"Adding category: {category}")
    df_new.loc[index,'Category'] = category
    #print(f"Adding room: {room}")
    df_new.loc[index,'Room'] = room
    

In [163]:
df_new.dtypes

Order Date      datetime64[ns, UTC]
Unit Price                  float64
Product Name                 object
ASIN                         object
Category                     object
Room                         object
dtype: object

In [141]:
df2 = df_brief_small.copy()

In [149]:
df2['Order Date'] = df2['Order Date'].str.replace(r'\.\d+', '', regex=True)

df2['Order Date 2'] = pd.to_datetime(df2['Order Date'], utc=True)

In [150]:
df2.head()

,Order Date,Unit Price,Product Name,ASIN,Order Date 2
315,2023-01-16T18:51:34Z,16.68,Scrubbing Bubbles Fresh Brush Flushables Refil...,B07YCZ3N36,2023-01-16 18:51:34+00:00
1106,2017-06-05T19:05:54Z,11.99,Choetech T511 7.5W Qi Wireless Charging Pad wi...,B00C40OG22,2017-06-05 19:05:54+00:00
999,2018-05-31T00:18:01Z,29.95,Antarctic Krill Oil (Double Strength) 1000mg p...,B00IP1E3O0,2018-05-31 00:18:01+00:00
752,2020-09-11T14:52:30Z,4.89,"TERRO T300 Liquid Ant Bait Ant Killer, 6 Bait ...",B000HJBKMQ,2020-09-11 14:52:30+00:00
1141,2017-02-09T23:07:28Z,184.95,TP-Link AC3150 Wireless Wi-Fi Gigabit Router w...,B01AK9TC0Y,2017-02-09 23:07:28+00:00


In [151]:
null_mask = df2.isnull().any(axis=1)
null_rows = df2[null_mask]

print(null_rows)

Empty DataFrame
Columns: [Order Date, Unit Price, Product Name, ASIN, Order Date 2]
Index: []


In [147]:
pd.to_datetime('2022-10-02T05:24:49.872Z')

Timestamp('2022-10-02 05:24:49.872000+0000', tz='UTC')

In [187]:
df_filtered.shape


(121, 4)

In [183]:
df_filtered.head()

,Order Date,Unit Price,Product Name,ASIN
0,2024-09-17 23:38:02+00:00,16.99,Blender Replacement 6 Fins for Ninja Blade 16 ...,B089QFC5QH
1,2024-09-16 00:30:45+00:00,80.85,Optimum Nutrition Gold Standard 100% Whey Prot...,B000QSTBNS
2,2024-09-14 17:20:46+00:00,17.04,Old Path White Clouds: Walking in the Footstep...,0938077260
3,2024-09-12 02:04:33+00:00,15.26,REACH Listerine Ultraclean Dental Floss 6-Pack...,B00LITWSW0
4,2024-09-11 20:09:36+00:00,7.13,"StarKist Chunk Light Tuna in Water, 5 oz (8 Pa...",B00FWUO2IE


In [192]:
df_new['Order Date'] = df_new['Order Date'].dt.tz_localize(None)


In [19]:
df_new.head()

,Order Date,Unit Price,Product Name,ASIN,Category,Room
0,2024-09-17 23:38:02+00:00,16.99,Blender Replacement 6 Fins for Ninja Blade 16 ...,B089QFC5QH,kitchen appliances,kitchen
1,2024-09-16 00:30:45+00:00,80.85,Optimum Nutrition Gold Standard 100% Whey Prot...,B000QSTBNS,sports nutrition,gym
2,2024-09-14 17:20:46+00:00,17.04,Old Path White Clouds: Walking in the Footstep...,0938077260,books,living room
3,2024-09-12 02:04:33+00:00,15.26,REACH Listerine Ultraclean Dental Floss 6-Pack...,B00LITWSW0,bath supplies,bathroom
4,2024-09-11 20:09:36+00:00,7.13,"StarKist Chunk Light Tuna in Water, 5 oz (8 Pa...",B00FWUO2IE,canned goods,pantry


In [194]:
# Write to file

df_new.to_csv('orders_out.csv', index=False)

In [25]:
df_to_store = df_new


In [26]:
df_to_store.head(10)

,Order Date,Unit Price,Product Name,ASIN,Category,Room
0,2024-09-17 23:38:02+00:00,16.99,Blender Replacement 6 Fins for Ninja Blade 16 ...,B089QFC5QH,kitchen appliances,kitchen
1,2024-09-16 00:30:45+00:00,80.85,Optimum Nutrition Gold Standard 100% Whey Prot...,B000QSTBNS,sports nutrition,gym
2,2024-09-14 17:20:46+00:00,17.04,Old Path White Clouds: Walking in the Footstep...,0938077260,books,living room
3,2024-09-12 02:04:33+00:00,15.26,REACH Listerine Ultraclean Dental Floss 6-Pack...,B00LITWSW0,bath supplies,bathroom
4,2024-09-11 20:09:36+00:00,7.13,"StarKist Chunk Light Tuna in Water, 5 oz (8 Pa...",B00FWUO2IE,canned food,pantry
5,2024-09-11 20:07:02+00:00,11.19,Kewpie DEEP-ROASTED SESAME DRESSING (16.9OZ),B08M9PR731,condiments,kitchen
6,2024-09-11 17:46:51+00:00,23.49,"Energizer 9V Batteries, Ultimate Lithium, Orig...",B01684J7P0,batteries,utility
7,2024-08-31 18:14:10+00:00,38.99,"FelizMax Crescent Zafu Meditation Pillow, 44x2...",B07QSTGBMM,bath supplies,living room
8,2024-08-31 16:01:40+00:00,10.44,"SKIPPY Creamy Peanut Butter, 5 Pound",B00LPE5EZQ,food,kitchen
9,2024-08-29 12:53:41+00:00,39.97,"Ultra Concentrated Organic Mushroom Coffee 3,5...",B0D41N4J1Y,beverages,kitchen


In [27]:
df_to_store.shape

(821, 6)